In [8]:
import pandas as pd

# Load the dataset (replace with your dataset file)
data = pd.read_csv("mypersonality_final.csv", encoding="latin1")

# Remove the first set of personality columns
data = data.drop(columns=["sEXT", "sNEU", "sAGR", "sCON", "sOPN"])

# Remove the additional columns
columns_to_remove = ["#AUTHID","NETWORKSIZE", "BETWEENNESS", "NBETWEENNESS", "DENSITY", "BROKERAGE", "NBROKERAGE", "TRANSITIVITY"]
data.drop(columns=columns_to_remove, inplace=True)

# Replace 'n' with 0 and 'y' with 1 for the remaining columns
columns_to_replace = ["cEXT", "cNEU", "cAGR", "cCON", "cOPN"]
data[columns_to_replace] = data[columns_to_replace].replace({'n': 0, 'y': 1})

# Remove the "c" from the columns "sEXT," "sNEU," "sAGR," "sCON," and "sOPN"
data.columns = data.columns.str.replace('c', '')

# Remove the "DATE" column
data = data.drop(columns=["DATE"])

# Print the modified dataset
print(data.head())

data.head(15)



                                              STATUS  EXT  NEU  AGR  CON  OPN
0                        likes the sound of thunder.    0    1    0    0    1
1  is so sleepy it's not even funny that's she ca...    0    1    0    0    1
2  is sore and wants the knot of muscles at the b...    0    1    0    0    1
3         likes how the day sounds in this new song.    0    1    0    0    1
4                                        is home. <3    0    1    0    0    1


,STATUS,EXT,NEU,AGR,CON,OPN
0,likes the sound of thunder.,0,1,0,0,1
1,is so sleepy it's not even funny that's she ca...,0,1,0,0,1
2,is sore and wants the knot of muscles at the b...,0,1,0,0,1
3,likes how the day sounds in this new song.,0,1,0,0,1
4,is home. <3,0,1,0,0,1
5,www.thejokerblogs.com,0,1,0,0,1
6,"saw a nun zombie, and liked it. Also, *PROPNAM...",0,1,0,0,1
7,is in Kentucky. 421 miles into her 1100 mile j...,0,1,0,0,1
8,was about to finish a digital painting before ...,0,1,0,0,1
9,is celebrating her new haircut by listening to...,0,1,0,0,1


In [10]:
import pandas as pd
from transformers import TFBertModel, BertTokenizer
import os
import re
import random
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt

In [11]:
# Set a seed value for reproducibility
seed_value = 29
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
# Check for GPU availability
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [13]:
# Define your axes and classes
N_AXIS = 5  # You have 5 axes: EXT, NEU, AGR, CON, and OPN
MAX_SEQ_LEN = 128
BERT_NAME = 'bert-base-uncased'

In [18]:
axes = ["EXT", "NEU", "AGR", "CON", "OPN"]

In [19]:
# Define the text preprocessing function
def text_preprocessing(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text.encode('ascii', 'ignore').decode('ascii')
    if text.startswith("'"):
        text = text[1:-1]
    return text

In [20]:
data = data.sample(frac=1)  # Shuffle the data

In [21]:
# Extract the "STATUS" column and personality traits
sentences = data["STATUS"]
personality_traits = ["EXT", "NEU", "AGR", "CON", "OPN"]
labels = data[personality_traits]